# Chapter 1. Systems, vectors and matrices

In [2]:
import numpy as np
import sys
sys.path.append("..")
import linalgkit as lak

## Gauss-Jordan method. Rouchè-Frobenious theorem

Consider a system of $m$ linear equations and $n$ unknowns,
$$
\left\{
\begin{array}{ccccccccc}
a_{11} x_1 &+& a_{12} x_2 &+& \cdots &+& a_{1n} x_n &=& b_1 \\
a_{21} x_1 &+& a_{22} x_2 &+& \cdots &+& a_{2n} x_n &=& b_2 \\
\vdots     & & \vdots     & &        & & \vdots     & & \vdots \\
a_{m1} x_1 &+& a_{m2} x_2 &+& \cdots &+& a_{mn} x_n &=& b_m \\
\end{array}
\right.
$$
There are two matrices associated with a system like the previous one:
$$\begin{array}{cc}
\text{Matriz de coeficientes} & \text{Matriz ampliada} \\[10pt]
    \left(\begin{array}{cccc}
    a_{11} & a_{12} & \cdots & a_{1n} \\
    a_{21} & a_{22} & \cdots & a_{2n} \\
    \vdots & \vdots & \ddots & \vdots \\
    a_{m1} & m_{m2} & \cdots & a_{mn}
    \end{array}\right)
    &
    \left(\begin{array}{cccc|c}
    a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
    a_{21} & a_{22} & \cdots & a_{2n} & b_2 \\
    \vdots & \vdots & \ddots & \vdots & \vdots \\
    a_{m1} & a_{m2} & \cdots & a_{mn} & b_m
    \end{array}\right) 
\end{array}$$

- The _coefficient matrix_ contains only the coefficients of the system.
- The _augmented matrix_ contains the coefficients and the independent terms (constants on the other side of the equation).

If the augmented matrix of the previous system is,

$$\left(\begin{array}{cccc|c}
    a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
    a_{21} & a_{22} & \cdots & a_{2n} & b_2 \\
           &        & \vdots &        & \vdots \\
    a_{m1} & a_{m2} & \cdots & a_{mn} & b_m
\end{array}\right)$$


> The _Gauss-Jordan reduction method_ (GJ) consists of reducing the augmented matrix of the system to its _reduced row echelon form_ using _elementary row operations_ (operations among the system's equations). From this last matrix, we can directly obtain the solution to the system, if it exists.

The reduced row echelon matrix is a matrix with the following characteristics:

The reduced row echelon matrix is a matrix with the following characteristics:

<p align="center">
    <img src="../figures/fig_reduced_matrix.svg" alt="Reduced row echelon matrix" width=20%/>
</p>

1. Each step has height one.
2. All numbers below the steps are zero.
3. In every corner of each step, there is a 1.
4. Every column containing a step has all its elements zero (except for the step itself, which is 1).

>In general, systems can be _consistent_ if they admit at least one solution, or _inconsistent_ if the system has no solution. If they are consistent, they are called _determined_ if they have a unique solution; or _undetermined_ if they have infinitely many solutions.

In the study of solutions to a system of equations with $m$ equations and $n$ unknowns, the steps obtained in the coefficient and augmented matrices (when we reduce them) are crucial. Let $A$ be the coefficient matrix of the system and $\bar{A}$ the augmented matrix. When we reduce the system using GJ, both matrices become row echelon: let $p$ and $\bar{p}$ be the steps of the reduced coefficient and augmented matrices, respectively. Then, to determine the compatibility of the system, it is only necessary to compare the number of unknowns $n$ with the steps $p$ and $\bar{p}$:

> Rouché–Frobenius Theorem
>
> Given a system of $m$ equations and $n$ unknowns, we have:
> 1. If $n = p = \bar{p} \quad \Leftrightarrow \quad$ The system is consistent and determined.
> 2. If $n > p = \bar{p} \quad \Leftrightarrow \quad$ The system is consistent and undetermined.
> 3. If $p \neq \bar{p} \quad \Leftrightarrow \quad$ The system is inconsistent.
>
The set of elementary operations we perform on the matrix may vary, but the row echelon matrix will always be the same (unique). This leads to multiple ways of reducing a system and, therefore, to multiple algorithms that reduce the matrix using different criteria, even though the same reduction method is followed.

The next example will discuss how to reduce a given matrix with the GJ method. We will go through the steps and explain the criteria chosen for the algorithm.


### Working example

Let's start reducing the following matrix:

In [3]:
matrix = np.array([[1, 2, 3, 4, 5],
                   [2, 4, 6, 8, 10],
                   [3, 8, 9, 12, 18],
                   [4, 11, 12, 17, 20]])

We have a $4 \times 5$ matrix from which we want to obtain its reduced row echelon form using Gauss–Jordan elimination (GJ). As previously mentioned, there are multiple ways to reduce a matrix, but the row echelon matrix obtained must be the same for everyone. Let's begin by looking at one of the possible ways to implement the GJ method.

First, take a copy of the original matrix on which we will perform elementary row operations:

In [4]:
# Copy of the matrix and shape
A = matrix.astype(float).copy()
m, n = matrix.shape

where we have changed the matrix datatype in order to perform operations.

The GJ method begins by taking the first row of the matrix: the first element of that row will be used to eliminate the elements below it. Let’s call `k` the index of the row we are focusing on, and `l` the index of the column where the first element of row `k` is located. These two indices will define each step of our algorithm.

#### Step `k = l = 0`


Obviously, at this step, we are taking the first row and column of the matrix, which can be schematically represented as follows:


```plaintext
            l = 0
              |
k = 0 ---> [[ 1  2  3  4  5]
            [ 2  4  6  8 10]
            [ 3  8  9 12 18]
            [ 4 11 12 17 20]]
```

These two indices represent the row `row = A[k, l:]` and the column `col = A[k:, l]` of `A`, which will be useful for the operations we are going to perform:

In [5]:
# Prepare row and column of the matrix
k, l = 0, 0
col = A[k:, l]
row = A[k, l:]

> **Note.** It is possible that the first row we take is not very convenient and we may want to pivot with another, more manageable one. In this case, the first row has a 1 in its first element, which is one of the most convenient situations: let’s leave the pivoting problem for a later step.

Now let’s perform the elementary row operations. To do this, we need to use the first row to eliminate the coefficients along column `l`. We can do this as follows:

In [6]:
print("Full matrix:")
print(A)

# Indices of the rows to be eliminated
idx = np.arange(m)
idx = idx[idx != k] # All rows except the one we are focusing on

# Subarray and coefficients
block = A[idx, l:]  # Subarray of the selected rows
coef = block[:, [0]]    # First coefficients of those rows

print("Block to eliminate:")
print(block)
print("First coefficients:")
print(coef)

# Operation
A[idx, l:] = block - coef * row # Elementary row operations

print("After the operation:")
print(A)

Full matrix:
[[ 1.  2.  3.  4.  5.]
 [ 2.  4.  6.  8. 10.]
 [ 3.  8.  9. 12. 18.]
 [ 4. 11. 12. 17. 20.]]
Block to eliminate:
[[ 2.  4.  6.  8. 10.]
 [ 3.  8.  9. 12. 18.]
 [ 4. 11. 12. 17. 20.]]
First coefficients:
[[2.]
 [3.]
 [4.]]
After the operation:
[[1. 2. 3. 4. 5.]
 [0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 3.]
 [0. 3. 0. 1. 0.]]


The final print shows that we succeed to eliminate the elements of the first column.

#### Step `k = l = 1` 

We now move to the second row and column of our matrix (`k = l = 1`). Again, schematically we would have:

```plaintext
             l = 1
               |
          [[1. 2. 3. 4. 5.]
k = 1 -->  [0. 0. 0. 0. 0.]
           [0. 2. 0. 0. 3.]
           [0. 3. 0. 1. 0.]]
```
We take the corresponding row and column:

In [7]:
k, l = 1, 1
col = A[k:, l]
row = A[k, l:]

print("Row:", row)
print("Column:", col)

Row: [0. 0. 0. 0.]
Column: [0. 2. 3.]


The row we are currently at is not suitable for performing operations (it's all zeros), so it will be necessary to pivot rows. The criterion we will use is the following:

> Among all the nonzero elements of the column `col = A[k:, l]`, we look for the row that contains the maximum in absolute value, and we pivot this row with row `k`.

In our case, the maximum value of the column `col` is `3`, which is found in row `3`, and it should be pivoted with row `k=1`. This can be done as follows:

In [8]:
# Indices of the nonzero elements
nonzero_idx = col.nonzero()[0]

# Index of the maximum absolute value in `col`
abs_col = np.abs(col)
max_nonzero_idx = nonzero_idx[np.argmax(abs_col[abs_col != 0])] 

# Row of the matrix where that value is found
row_idx = max_nonzero_idx + k

print("Current row:")
print(row)
print("Row to pivot with:")
print(A[row_idx, l:])

# Pivot
A[[k, row_idx], l:] = A[[row_idx, k], l:]
print("Matrix after pivoting:")
print(A)

Current row:
[0. 0. 0. 0.]
Row to pivot with:
[3. 0. 1. 0.]
Matrix after pivoting:
[[1. 2. 3. 4. 5.]
 [0. 3. 0. 1. 0.]
 [0. 2. 0. 0. 3.]
 [0. 0. 0. 0. 0.]]


Once we have pivoted the rows, it is convenient to turn the first element of row `k` into 1. This is commonly known as _normalization_ of the first element:

In [9]:
# Normalization of the first element
A[k, l:] = row / row[0]

Know we perform matrix operations between rows:

In [10]:
print("Full matrix:")
print(A)

# Indices of the rows to be eliminated
idx = np.arange(m)
idx = idx[idx != k] # All rows except the one we are focusing on

# Subarray and coefficients
block = A[idx, l:]  # Subarray of the selected rows
coef = block[:, [0]]    # First coefficients of those rows

print("Block to eliminate:")
print(block)
print("First coefficients:")
print(coef)

# Operation
A[idx, l:] = block - coef * row # Elementary row operations

print("After the operation:")
print(A)

Full matrix:
[[1.         2.         3.         4.         5.        ]
 [0.         1.         0.         0.33333333 0.        ]
 [0.         2.         0.         0.         3.        ]
 [0.         0.         0.         0.         0.        ]]
Block to eliminate:
[[2. 3. 4. 5.]
 [2. 0. 0. 3.]
 [0. 0. 0. 0.]]
First coefficients:
[[2.]
 [2.]
 [0.]]
After the operation:
[[ 1.          0.          3.          3.33333333  5.        ]
 [ 0.          1.          0.          0.33333333  0.        ]
 [ 0.          0.          0.         -0.66666667  3.        ]
 [ 0.          0.          0.          0.          0.        ]]


#### Step `k = l = 2`

We move to column and row `k = l = 2`. Schematically, we have:

```plaintext
                              l = 2
                                |
     [[ 1.          0.          3.          3.33333333  5.        ]
      [ 0.          1.          0.          0.33333333  0.        ]
k = 2 [ 0.          0.          0.         -0.66666667  3.        ]
      [ 0.          0.          0.          0.          0.        ]]
```

In this case, the column `col = A[k:, l]` is already full of zeros, so no operation is needed. The next step is to move one unit to the right in the column, without moving from row `k`. Since this is supposed to be an iterative algorithm, at each step we should check if our column is full of zeros:

In [11]:
k, l = 2, 2

col = A[k:, l]
row = A[k, l:]

# Nonzero elements
nonzero_idx = col.nonzero()[0]

# Check if columns is full of zeros
nonzero_idx.size == 0

True

In the loop we define, this will be checked at each step. If `nonzero_idx.size == 0` holds, we move one unit to the right in the column (`l += 1`) and return to the beginning; otherwise, we continue as before.

#### Step `k =2, l =3`

We are now at row `k = 2` and column `l = 3`:

```plaintext
                                              l = 3
                                                |
     [[ 1.          0.          3.          3.33333333  5.        ]
      [ 0.          1.          0.          0.33333333  0.        ]
k = 2 [ 0.          0.          0.         -0.66666667  3.        ]
      [ 0.          0.          0.          0.          0.        ]]

```
We take the corresponding row and column:

In [12]:
k, l = 2, 3

col = A[k:, l]
row = A[k, l:]

print("Column:", col)
print("Row:", row)

Column: [-0.66666667  0.        ]
Row: [-0.66666667  3.        ]


In this case, we do not need to pivot the row. Normalizing the row:

In [13]:
# Normalization of the first element
A[k, l:] = row / row[0]

Performing operations:

In [14]:
print("Full matrix:")
print(A)

# Indices of the rows to be eliminated
idx = np.arange(m)
idx = idx[idx != k] # All rows except the one we are focusing on

# Subarray and coefficients
block = A[idx, l:]  # Subarray of the selected rows
coef = block[:, [0]]    # First coefficients of those rows

print("Block to eliminate:")
print(block)
print("First coefficients:")
print(coef)

# Operation
A[idx, l:] = block - coef * row # Elementary row operations

print("After the operation:")
print(A)

Full matrix:
[[ 1.          0.          3.          3.33333333  5.        ]
 [ 0.          1.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          1.         -4.5       ]
 [ 0.          0.          0.          0.          0.        ]]
Block to eliminate:
[[3.33333333 5.        ]
 [0.33333333 0.        ]
 [0.         0.        ]]
First coefficients:
[[3.33333333]
 [0.33333333]
 [0.        ]]
After the operation:
[[ 1.   0.   3.   0.  20. ]
 [ 0.   1.   0.   0.   1.5]
 [ 0.   0.   0.   1.  -4.5]
 [ 0.   0.   0.   0.   0. ]]


The matrix is fully reduced, so we are mostly finished.

#### Last step

Once we have the reduced matrix, we need to count the steps (pivots) of the coefficient and augmented matrices. We can do this by looking at the indices of the nonzero elements remaining in the reduced matrix:

In [15]:
# Number of steps (pivots) in the augmented matrix
aug_s = np.unique(A.nonzero()[0]).size 

# Number of steps (pivots) in the coefficient matrix
coef_s = np.unique(A[:, :-1].nonzero()[0]).size 

print(aug_s, coef_s)

3 3


Both steps are equal, but less than the number of unknowns (4): the system would be consistent and undetermined.

### Fine-tuning Gauss-Jordan Method

The previous steps suffers from the following:

> Due to limited float precision when writting a number, it is not a good practice to compare elements of the matrix directly with 0. Instead we should consider introducing some kind of tolerance, so that we can "set to zero" the numbers below this value.

The tolerance value we should use depends of the data's context. For example, it is not the same matrix data that comes from experimental measures with a certain precision value, to data with no "experimental" precision associated but subject to float number precision. Is up to the user to determine which tolerance adjust better to the situation (i.e. the tolerance value that gives better results).

Let's call `tol` the _tolerance_ value that allow us to adjust our method. It is direct and simple to introduce a comparison of elements with this value: we just habe to set to zero every element below `tol` on every single `(k, l)` step. In other words, we just have to introduce this line on each step:

```python
# ---- Rest of code -----

A[np.abs(A) <= tol] = 0.

#------ Rest of code -----
```
With all this in mind, we are good to go to see the final implemetation.

### Implementation

The previous example is a fairly complete case of the steps we may encounter when reducing a matrix: the implementation is straightforward if we follow the procedure above.

In the `linalkit` package, we can find the `gj_reduction` function with its source code. This function performs Gauss-Jordan reduction on a given matrix. We present it below:

| Function      | Parameters                                                                                                                                         | Return                                                                                                                                                                                                                                                                         | Notes                                                                                                                                                                                                                          |
|---------------|----------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `gj_reduction` | - `matrix`: 2D matrix to reduce<br>- `give_syspar`: bool, if `True` returns system parameters (`False` by default)<br>- `tol`: float, tolerance for zeroing small values (`1e-12` by default) | - If `give_syspar` is `False`:<br> • reduced matrix<br>- If `True`: a tuple with:<br> • reduced matrix<br> • number of unknowns<br> • rank (stairs) of the augmented matrix<br> • rank (stairs) of the coefficient matrix | - The last column is interpreted as constants if the matrix represents a system of equations.<br>- The "stairs" (ranks) help analyze system compatibility.<br>- Can be used with any matrix, not necessarily a system.<br>- `tol` helps avoid numerical precision issues. |

If we apply this function to the previous example we obtain:

In [16]:
lak.gj_reduction(matrix, give_syspar=True)

(array([[ 1. ,  0. ,  3. ,  0. , 20. ],
        [ 0. ,  1. ,  0. ,  0. ,  1.5],
        [ 0. ,  0. ,  0. ,  1. , -4.5],
        [ 0. ,  0. ,  0. ,  0. ,  0. ]]),
 4,
 3,
 3)

The final result is a tuple that contains the number of unknowns and the stairs of the coefficient and augmented matrices.

## Linear Dependence and Rank of Vectors

In the context of systems of linear equations and matrices, the concept of a vector is often discussed. A _vector_ $\vec{a}$ represents an ordered list of numbers in the following form:

$$ \vec{a} = (a_1, a_2, \dots, a_n) $$

- Each of the numbers that make up the vector $\vec{a}$ is called a _component_. The value $a_1$ is the _first_ component; $a_2$ the _second_ component; and so on. In this case, $\vec{a}$ is a vector with $n$ components.

- Two vectors $\vec{a}$ and $\vec{b}$ with $n$ components are said to be _equal_ if they are equal component by component. That is,
$$\vec{a} = \vec{b} \quad \Leftrightarrow \quad a_1 = b_1, a_2 = b_2, \dots, a_n = b_n$$ 

We define two operations on vectors:
- **Addition**

    Given two vectors $\vec{a} = (a_1, a_2, \dots, a_n)$ and $\vec{b} = (b_1, b_2, \dots, b_n)$, their sum is the vector:

    $$
    \vec{a} + \vec{b} = (a_1 + b_1,\, a_2 + b_2,\, \dots,\, a_n + b_n)
    $$

- **Scalar multiplication**

    Given a scalar $\lambda$ and a vector $\vec{a} = (a_1, a_2, \dots, a_n)$, the product is:

    $$
    \lambda \vec{a} = (\lambda a_1,\, \lambda a_2,\, \dots,\, \lambda a_n)
    $$

These two operations satisfy the following properties related to associativity and distributivity:

| Property                                         | Mathematical expression                                      | Name/Description                                 |
|--------------------------------------------------|--------------------------------------------------------------|--------------------------------------------------|
| Associativity of addition                        | $\vec{a} + (\vec{b} + \vec{c}) = (\vec{a} + \vec{b}) + \vec{c}$ | Vector addition is associative                   |
| Commutativity of addition                        | $\vec{a} + \vec{b} = \vec{b} + \vec{a}$                     | Vector addition is commutative                   |
| Additive identity                                | $\vec{a} + \vec{0} = \vec{a}$                               | There exists a zero vector that does not alter the sum |
| Additive inverse                                 | $\vec{a} + (-\vec{a}) = \vec{0}$                            | Every vector has an additive inverse             |
| Associativity of scalar multiplication           | $\lambda(\mu\vec{a}) = (\lambda\mu)\vec{a}$                  | Scalar multiplication is associative             |
| Multiplicative identity for scalars              | $1\cdot\vec{a} = \vec{a}$                                   | 1 is the identity for scalar multiplication      |
| Distributivity over vector addition              | $\lambda(\vec{a} + \vec{b}) = \lambda\vec{a} + \lambda\vec{b}$ | Scalar multiplication distributes over vector addition |
| Distributivity over scalar addition              | $(\lambda + \mu)\vec{a} = \lambda\vec{a} + \mu\vec{a}$      | Scalar addition distributes over vector multiplication |

Vectors can be found, for example, in the rows and columns of a matrix. Given a matrix $A$ of size $m \times n$ of the form 

$$
A = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
$$

its rows and columns form vectors of $n$ and $m$ components, respectively. The row vectors can be denoted as:
$$ 
\begin{aligned}
    &\vec{r}_1 = \begin{pmatrix} a_{11} & a_{12} & \dots & a_{1n} \end{pmatrix} \\
    &\vec{r}_2 = \begin{pmatrix} a_{21} & a_{22} & \dots & a_{2n} \end{pmatrix} \\
    & \qquad \qquad \qquad \vdots \\
    &\vec{r}_m = \begin{pmatrix} a_{m1} & a_{m2} & \dots & a_{mn} \end{pmatrix}
\end{aligned}
$$
and the column vectors as:
$$
\vec{c}_1 = \begin{pmatrix} a_{11} \\ a_{21} \\ \vdots \\ a_{m1} \end{pmatrix}, \quad
\vec{c}_2 = \begin{pmatrix} a_{12} \\ a_{22} \\ \vdots \\ a_{m2} \end{pmatrix}, \quad
\cdots, \quad
\vec{c}_n = \begin{pmatrix} a_{1n} \\ a_{2n} \\ \vdots \\ a_{mn} \end{pmatrix}
$$

We will also see that the solution to a system of equations can be given in vector notation.

With all this in mind, we can move on to study the concept of linear dependence and rank among vectors.n el contexto de sistemas de ecuaciones lineales y matrices, se suele hablar del concepto de vector. 


### Linear Dependence

Consider a set of $n$ vectors $\left\{ \vec{a}_i \right\}_{i=1}^n$ of $m$ components. We say we perform a _linear combination_ of the vectors if we have an expression of the form
$$c_1 \vec{a}_1 + c_2 \vec{a}_2 + \cdots + c_n \vec{a}_n, \quad c_i \in \mathbb{R}\ (i=1,\dots,n)$$
which is simply a combined operation of the given vectors that generates a new vector.

In this context, we usually encounter two situations for our set of vectors.

> 1. Linear dependence
>
>     A set of vectors $\left\{ \vec{a}_i \right\}_{i=1}^n$ is said to be _linearly dependent_ if there exist $n$ real values $c_i$ (not all zero) such that 
> $$c_1 \vec{a}_1 + c_2 \vec{a}_2 + \cdots + c_n \vec{a}_n = \vec{0}$$
>
> 2. Linear independence
>
>       A set of vectors $\left\{ \vec{a}_i \right\}_{i=1}^n$ is said to be _linearly independent_ if it is not linearly dependent. That is,
> $$\text{If}\quad c_1 \vec{a}_1 + c_2 \vec{a}_2 + \cdots + c_n \vec{a}_n = \vec{0} \quad \Rightarrow \quad c_i=0 \quad \forall i=1,\dots,n$$

Checking whether a set of vectors is linearly dependent or not is quite straightforward according to this definition. For a set of $n$ vectors of $m$ components:

1. **State the condition for linear dependence**

    That is, set up a zero linear combination of the vectors as
    $$c_1 \vec{a}_1 + c_2 \vec{a}_2 + c_3 \vec{a}_3 + \cdots + c_n \vec{a}_n = \vec{0}$$
    From here, it is more convenient to write everything in matrix notation. Thus, the above is equivalent to writing:
    $$
    c_1
    \begin{pmatrix}
    a_{11} \\ a_{21} \\ \vdots \\ a_{m1}
    \end{pmatrix}
    + c_2
    \begin{pmatrix}
    a_{12} \\ a_{22} \\ \vdots \\ a_{m2}
    \end{pmatrix}
    + \cdots +
    c_n
    \begin{pmatrix}
    a_{1n} \\ a_{2n} \\ \vdots \\ a_{mn}
    \end{pmatrix}
    =
    \begin{pmatrix}
    c_1 a_{11} + c_2 a_{12} + \cdots + c_n a_{1n} \\
    c_1 a_{21} + c_2 a_{22} + \cdots + c_n a_{2n} \\
    \vdots \\
    c_1 a_{m1} + c_2 a_{m2} + \cdots + c_n a_{mn}
    \end{pmatrix}
    =
    \begin{pmatrix}
    0 \\ 0 \\ \vdots \\ 0
    \end{pmatrix}
    $$
    Equating component by component, this results in a homogeneous system of $m$ equations and $n$ unknowns (the values $c_i$):
    $$
    \begin{cases}
    a_{11} c_1 + a_{12} c_2 + \cdots + a_{1n} c_n = 0 \\
    a_{21} c_1 + a_{22} c_2 + \cdots + a_{2n} c_n = 0 \\
    \qquad \qquad \qquad \quad \vdots \\
    a_{m1} c_1 + a_{m2} c_2 + \cdots + a_{mn} c_n = 0
    \end{cases}
    $$

2. **Reduce the system**

    The matrix of the previous system is easy to obtain directly. Note that it is enough to place the vectors $\vec{a}_i$ in columns, so we would have to reduce the matrix
    $$
    \begin{pmatrix}
    a_{11} & a_{12} & \cdots & a_{1n} \\
    a_{21} & a_{22} & \cdots & a_{2n} \\
    \vdots & \vdots &        & \vdots \\
    a_{m1} & a_{m2} & \cdots & a_{mn}
    \end{pmatrix}
    $$
     Here we can have only two possible cases:

    - The system is consistent and determined. In this case, the only solution is that all $c_i$ are zero.

    - The system is consistent and undetermined. There exist nonzero solutions for the coefficients $c_i$.

3. **Interpretation/Conclusion**

    - If the system is determined with solution $c_i=0$, this means the vectors are linearly independent.

    - If the system is undetermined, there exist nonzero solutions for the $c_i$, in which case the vectors are linearly dependent.

This reasoning suggests using the `gj_reduction` function again to simply distinguish whether the system generated by the vectors is consistent and determined or not. 

The next example serves to clarify the intentions mentioned.


#### Working example

We want to check the linear dependence of the following set of vectors:

In [17]:
# Linearly dependent set of vectors
v1 = np.array([1, 2, 3])
v2 = np.array([2, 4, 6])
v3 = np.array([3, 6, 9])
v4 = np.array([4, 5, 6])

vectors_set = (v1, v2, v3, v4)

The above is a set of four linearly dependent vectors. To check this, we construct the matrix associated with the system generated by the vectors:

In [18]:
# Matrix with the vectors in columns

# Set size and vectors size
n = len(vectors_set)
m = len(v1)

# Build matrix
matrix = np.empty((m, n))
k = 0
for v in vectors_set:
    matrix[:, k] = v
    k += 1
    
matrix

array([[1., 2., 3., 4.],
       [2., 4., 6., 5.],
       [3., 6., 9., 6.]])

Finally we apply the `gj_reduction` to study the solution of the system:

In [19]:
lak.gj_reduction(matrix, give_syspar=True)

(array([[1., 2., 3., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]]),
 3,
 2,
 1)

Here we have to keep in mind a small detail: the function is designed to receive the augmented matrix of the system, and we have only provided the coefficient matrix. This does not pose any problem in the final reduction, because the system of vectors is homogeneous, and the last column consists of zeros. However, now the number of steps (pivots) we are interested in is that of the complete (augmented) matrix; that is, we have to look at the value of `aug_s` and compare it with `unknowns + 1`. The value of `coef_s` does not make sense in this case.

For this reason, to avoid this distinction, it is better to add a column of zeros to our system matrix, so that the function's values all have their proper meaning.

In [20]:
matrix = np.hstack((matrix, np.zeros(m)[:, np.newaxis]))
matrix

array([[1., 2., 3., 4., 0.],
       [2., 4., 6., 5., 0.],
       [3., 6., 9., 6., 0.]])

In [21]:
lak.gj_reduction(matrix, give_syspar=True)

(array([[1., 2., 3., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]]),
 4,
 2,
 2)

Now the number of steps `exp_s` and `coef_s` match, as they should (homogeneous systems are always consistent). So, we just need to compare the number of unknowns with the number of steps: in our case, $4 = n > p = 2$, so the system is underdetermined. We conclude that the vectors are linearly dependent.



### Rank of a set of vectors

The next natural step after studying linear dependence among vectors is to determine how many and which of them are linearly independent. This way, we will have all the information about the set: some vectors will be independent, and others will be generated by the others.

What we want to study is, precisely, the concept of rank.

> Rank of a set of vectors
>
> Given a set of $n$ vectors with $m$ components $S = \left\{\vec{a}_i\right\}_{i=1}^n$, we call the _rank of the set of vectors_, denoted by $r(S)$, the maximum number of linearly independent vectors among them.

How do we find the rank of a set of vectors? In the previous section, we already saw how to study the dependence of a set of vectors. To do this, we had to:

- Write the matrix generated by the vectors (colocating them in columns).
- Reduce the previous matrix.
- Study the compatibility and draw a conclusion.

If we do this for all the subsets of vectors we can form, we would eventually find the rank. This can be quite lengthy even for relatively small sets. Fortunately, we will now see that it is only necessary to study the dependence of the complete set once.

Let's start by considering the matrix formed by the set of vectors (when we pose the linear dependence with all of them)
$$
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots &        & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
$$
According to what we said before, we would proceed as follows:

- Reduce the complete matrix
    - If it is determined, the rank is $n$
    - If it is undetermined, the rank is less than $n$ and we move to the next step.
- Reduce all the submatrices of $n-1$ vectors
    - If any of them is determined, the rank is $n-1$
    - If it is undetermined, the rank is less than $n-1$ and we move to the next step.
- We repeat this process indefinitely until we obtain the first subset that is linearly independent.

All this process can be done just by reducing the complete matrix. The key here is to realize that the operations we have performed on the complete matrix are the same as those we would have to do on any subset we consider from the matrix. Therefore, if we want to know the reduced matrix of a particular subset of vectors, we just have to look for the columns associated with those vectors in the fully reduced matrix: at a glance, we can see if those vectors produce a determined or undetermined system.

If we take a look at the fully reduced matrix

<p align="center">
    <img src="../figures/fig_reduced_matrix.svg" alt="Reduced row echelon matrix" width=25%/>
</p>

we can directly see that if we choose the vectors (columns) that form the pivots (steps) of the matrix, we directly obtain a determined system: those vectors would be linearly independent. This subset would not be the only linearly independent one; we can find more vectors that generate a determined system, but at most, the rank we obtain will be equal to the number of pivots in the matrix.

> The rank of a set of vectors matches the number of pivots (steps) in the reduced matrix generated by those vectors.

- The independent vectors are obtained by choosing the columns of the reduced matrix that generate a determined system. Specifically, the simplest linearly independent set is formed by the vectors (columns) that contain the pivots of the reduced matrix.

We now have the procedure to follow to obtain the rank of a set of vectors and a linearly independent subset.

#### Working example

Let's consider the same example of the last section:

In [22]:
v1 = np.array([1, 2, 3])
v2 = np.array([2, 4, 6])
v3 = np.array([3, 6, 9])
v4 = np.array([4, 5, 6])

vectors_set = (v1, v2, v3, v4)

Ya comprobamos en el apartado anterior que este conjunto de vectores es linealmente dependiente:

In [ ]:
check_ld(*vectors_set)

Linearly dependent set


Ahora queremos obtener el rango de este conjunto y un subconjunto linealmente independiente. Para esto podemos usar de nuevo la información que nos proporciona la función `gj_reduction`:

- De la matriz reducida obtenemos el número de peldaños, que sería el rango.
- De la matriz reducida también podemos obtener las columnas que continen los peldaños.

Veamos cómo se realiza esto:

In [ ]:
# Construimos el sistema homogéneo generado por los vectores
m, n = len(vectors_set[0]), len(vectors_set)
matrix = np.zeros((m, n+1))
for k, v in enumerate(vectors_set):
    matrix[:, k] = v
print("Sistema generado por los vectores: \n", matrix)

# Reducimos el sistema por GJ
A, unknows, stairs, s = gj_reduction(matrix)
print("Sistema reducido: \n", A)

# 1. Guardamos el valor del rango

r = stairs

# 2. Buscamos las columnas que continen los peldaños

nonzero_idx = np.asarray(A.nonzero())
stairs_idx = np.array([nonzero_idx[1, 0]] + [ 
    nonzero_idx[1, k+1]
    for k in range(len(nonzero_idx[0]) - 1)
    if nonzero_idx[0][k+1] != nonzero_idx[0][k]
    ])  # Identificamos cuando cambia el índice de las filas para guardar el de las columnas

# 3. Damos el conjunto de vectores linealmente independiente
indp_set = np.asarray(vectors_set)[stairs_idx]
indp_set = tuple(indp_set)

print("Rango:", r)
print("Conjunto independiente: \n", indp_set)

Sistema generado por los vectores: 
 [[1. 2. 3. 4. 0.]
 [2. 4. 6. 5. 0.]
 [3. 6. 9. 6. 0.]]
Sistema reducido: 
 [[ 1.  2.  3.  0.  0.]
 [ 0.  0.  0.  1. -0.]
 [ 0.  0.  0.  0.  0.]]
Rango: 2
Conjunto independiente: 
 (array([1, 2, 3]), array([4, 5, 6]))


Definamos la función que sigue este procedimiento:

In [ ]:
def vectors_range(*vectors_set: np.ndarray, give_set: bool=False) -> tuple:
    """
    Gives the range of a given vector set, i.e. the maximum number of independent 
    vectors. If specified, it also gives an independent subset of vectors of size
    equal the range of the set.

    Parameters
    ----------
    `vectors_set`: array_like
        1D arrays with the vectors we want to study

    `give_set`: bool
        If set to True, it gives an independent subset of vector of size equal the
        range of the set. Default False.

    Returns
    -------
    A tuple with the following elements:

    `r`: int
        The range value of the vectors set.

    `indp_set`: tuple
        A tuple with the independent subset of vectors of size equal the range.
    """

    # We build the homogeneous system generated by the vectors set
    m, n = len(vectors_set[0]), len(vectors_set)
    matrix = np.zeros((m, n+1))
    for k, v in enumerate(vectors_set):
        matrix[:, k] = v

    # Reduce the matrix
    A, unknows, stairs, s = gj_reduction(matrix)

    # 1. Save range value

    r = stairs

    # 2. Stairs's columns

    nonzero_idx = np.asarray(A.nonzero())
    stairs_idx = np.array([nonzero_idx[1, 0]] + [ 
        nonzero_idx[1, k+1]
        for k in range(len(nonzero_idx[0]) - 1)
        if nonzero_idx[0][k+1] != nonzero_idx[0][k]
        ])  

    # 3. Indpendent set if needed
    if give_set == True:
        indp_set = np.asarray(vectors_set)[stairs_idx]
        indp_set = tuple(indp_set)
        return r, indp_set
    else:
        return r

Aplicado al mismo ejemplo anterior obtenemos:

In [ ]:
vectors_range(*vectors_set, give_set=True)

(2, (array([1, 2, 3]), array([4, 5, 6])))

### Rango de una matriz

Pasemos ahora a específicar qué es el rango de una matriz.

> Rango de una matriz
> 
> El _rango de una matriz_ $A$, denotado por $r(A)$, es el rango de sus vectores columna.

Por tanto, podemos utilizar todo lo aprendido en el apartado anterior para calcular el rango de una matriz. Tan solo habría que aplicarlo a sus vectores columna.

Consideremos la siguiente matriz:

In [ ]:
matrix = np.array([[1, 2, 3, 4],
                   [0, 1, 1, 1],
                   [1, 4, 5, 6]])
matrix

array([[1, 2, 3, 4],
       [0, 1, 1, 1],
       [1, 4, 5, 6]])

Tomemos sus vectores columna y planteemos su dependencia lineal a través de su sistema homogéneo asociado. En este caso, los vectores ya están en columnas!! Solo queda reducir la matriz y dar el valor del rango:

In [ ]:
m, n = matrix.shape
matrix = np.hstack((matrix, np.zeros(m)[:, np.newaxis]))
A, unknows, stairs, s = gj_reduction(matrix)
print("Matriz reducida: \n", A)

# Rango
r = stairs

print("Rango:", r)

Matriz reducida: 
 [[1. 0. 1. 2. 0.]
 [0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0.]]
Rango: 2


También podemos dar una submatriz cuyo rango sea el anterior tomando las columnas de los peldaños:

In [ ]:
nonzero_idx = np.asarray(A.nonzero())
stairs_idx = np.array([nonzero_idx[1, 0]] + [ 
    nonzero_idx[1, k+1]
    for k in range(len(nonzero_idx[0]) - 1)
    if nonzero_idx[0][k+1] != nonzero_idx[0][k]
    ])

submatrix = matrix[:, stairs_idx]
submatrix

array([[1., 2.],
       [0., 1.],
       [1., 4.]])

En el contexto de rango de matrices, sin embargo, es más común hablar del rango de los vectores fila de una matriz, porque suelen representar sistemas de ecuaciones para los que queremos saber cuales son linealmente independientes. Casualmente, da igual si calculas el rango de las filas o las columnas de una matriz, porque acabas obteniendo el mismo número.

> El rango de los vectores fila de una matriz coincide con el rango de los vectores columna.

Sin entrar en mucho más detalle, pensar que cuando estudiamos el rango de los vectores columna, y reducimos la matriz, estamos realizando operaciones elementales entre las filas: estos son, combinaciones lineales de las filas. Algunas de ellas se anulan (filas dependientes), mientras que otras sobreviven (filas independientes) formando los peldaños de la matriz reducida. Es por esto que acabamos obteniendo el mismo rango que los vectores columna.

Por completitud, demos ahora la función que calcula el rango de una matriz:

In [ ]:
def matrix_range(matrix: np.ndarray, give_submatrix: bool=False) -> tuple:
    """
    Returns the range of the given matrix. If specified, it also gives a
    submatrix with independent columns with the same range.

    Parameters
    ----------
    `matrix`: ndarray
        2D array representing the matrix we want to study.
    `give_submatrix`: bool
        If set to True, it gives gives a submatrix with independent columns 
        with same range. Default False.

    Returns
    -------
    A tuple with the following objects:

    `r`: int
        Range value of the matrix.
    `submatrix`: ndarray
        2D array of independent columns of the matrix.
    """
    
    # Reduce the given matrix
    m, n = matrix.shape
    matrix = np.hstack((matrix, np.zeros(m)[:, np.newaxis]))
    A, unknows, stairs, s = gj_reduction(matrix)

    # Give range
    r = stairs

    # Give submatrix
    if give_submatrix == True:
        nonzero_idx = np.asarray(A.nonzero())
        stairs_idx = np.array([nonzero_idx[1, 0]] + [ 
            nonzero_idx[1, k+1]
            for k in range(len(nonzero_idx[0]) - 1)
            if nonzero_idx[0][k+1] != nonzero_idx[0][k]
            ])
        submatrix = matrix[:, stairs_idx]
        return r, submatrix
    else:
        return r


Aplicado al ejemplo anterior:

In [ ]:
matrix = np.array([[1, 2, 3, 4],
                   [0, 1, 1, 1],
                   [1, 4, 5, 6]])

matrix_range(matrix, give_submatrix=True)

(2,
 array([[1., 2.],
        [0., 1.],
        [1., 4.]]))

Es posible que no nos interese conocer la submatriz de columnas linealmente independientes, sino las filas. En este caso, podríamos quedarnos con las filas que sobreviven después de reducir la matriz. Sin embargo, este enfoque requiere seguir las filas que sobreviven en cada paso que demos en la reducción: si pivotamos las filas, su posición final no se corresponderá con la inicial en general. Para evitar esto, sencillamente trasponemos la matriz y realizamos un estudio idéntico sobre esta matriz:

In [ ]:
# Trasponemos la matriz
row_matrix = matrix.T
row_matrix = np.hstack((row_matrix, np.zeros(len(row_matrix))[:, np.newaxis]))

# Reducimos la matriz
A, unknows, stairs, s = gj_reduction(row_matrix)

# Damos las filas linealmente independientes 
nonzero_idx = np.asarray(A.nonzero())
stairs_idx = np.array([nonzero_idx[1, 0]] + [ 
nonzero_idx[1, k+1]
for k in range(len(nonzero_idx[0]) - 1)
if nonzero_idx[0][k+1] != nonzero_idx[0][k]
])
indp_rows = row_matrix[:, stairs_idx].T

print("Filas linealmente independientes:")
print(row_matrix)
print(indp_rows)

Filas linealmente independientes:
[[1. 0. 1. 0.]
 [2. 1. 4. 0.]
 [3. 1. 5. 0.]
 [4. 1. 6. 0.]]
[[1. 2. 3. 4.]
 [0. 1. 1. 1.]]


Este caso sugiere ampliar el alcance de la definición anterior:

In [ ]:
def matrix_range(matrix: np.ndarray, give_cols: bool=False, give_rows: bool=False) -> tuple:
    """
    Returns the range of the given matrix. If specified, it also gives the independent
    columns and rows of the matrix.

    Parameters
    ----------
    `matrix`: ndarray
        2D array representing the matrix we want to study.
    `give_cols, give_rows`: bool
        If set to True, they give the independent columns and rows of the matrix,
        respectively. Default False.

    Returns
    -------
    A tuple with the following objects:

    `r`: int
        Range value of the matrix.
    `indp_cols, indp_rows`: ndarray
        2D array of independent columns and rows of the matrix, respectively.
    """
    
    # Reduce the given matrix
    m, n = matrix.shape
    aux_matrix = np.hstack((matrix, np.zeros(m)[:, np.newaxis]))
    A, _, stairs, _ = gj_reduction(aux_matrix)

    # Give range
    r = stairs

    results = [r]

    # Independent columns
    if give_cols:
        nonzero_idx = np.asarray(A.nonzero())
        stairs_idx = np.array([nonzero_idx[1, 0]] + [
            nonzero_idx[1, k+1]
            for k in range(len(nonzero_idx[0]) - 1)
            if nonzero_idx[0][k+1] != nonzero_idx[0][k]
        ])
        indp_cols = matrix[:, stairs_idx]
        results.append(indp_cols)

    # Independent rows
    if give_rows:
        aux_matrix = matrix.T
        aux_matrix = np.hstack((row_matrix, np.zeros(len(row_matrix))[:, np.newaxis]))
        A, _, _, _ = gj_reduction(row_matrix)
        nonzero_idx = np.asarray(A.nonzero())
        stairs_idx = np.array([nonzero_idx[1, 0]] + [
            nonzero_idx[1, k+1]
            for k in range(len(nonzero_idx[0]) - 1)
            if nonzero_idx[0][k+1] != nonzero_idx[0][k]
        ])
        indp_rows = aux_matrix[:, stairs_idx].T
        results.append(indp_rows)
    
    # Return only range value if no extra info requested, else tuple
    if len(results) == 1:
        return r
    else:
        return tuple(results)


Ahora sí tenemos una versión completa de la función anterior. Aplicada al mismo ejemplo:

In [ ]:
matrix = np.array([[1, 2, 3, 4],
                   [0, 1, 1, 1],
                   [1, 4, 5, 6]])

matrix_range(matrix, give_cols=True, give_rows=True)

(2,
 array([[1, 2],
        [0, 1],
        [1, 4]]),
 array([[1., 2., 3., 4.],
        [0., 1., 1., 1.]]))

Evidentemente, los conjuntos de columnas y filas proporcionados por la función no son los únicos. Pueden haber otra combinación de los vectores que también sea linealmente independiente.


Cómo se ha visto, `gj_reduction` es un algoritmo secuencial, por lo que sufrirá de los inconvenientes comunes de estos algoritmos. Sin embargo, representa una primera solución bastante útil para resolver sistemas sencillos: particularmente de números enteros. En el caso que se quiera resolver sistemas con números decimales (con alguna precisión) este algoritmo no es del todo conveniente, ya que haría falta _normalizar_ la matriz en cada paso: esto es, llevar a `0` todos aquellos valores que estén por debajo de la precisión de nuestros datos a medida que realizamos operaciones. Esto ocurre cuando se trabaja con datos que difieren en pocos decimales, o cuando los valores son muy cercanos a cero.

Para tener esto en cuenta, realizamos las modificaciones necesarias en la función anterior:

In [17]:
## Fine-tuning Gauss-Jordan reduction algorithm

def ftgj_reduction(coef_matrix: np.ndarray, values: np.ndarray, tol=float) -> tuple:
    '''
    Uses Gauss-Jordan reduction method to reduce a linear equations system with coefficient matrix 
    `coef_matrix` (2-D) and independent values `values` (1-D). The parameter `tol` adjusts the
    precission of our data: i.e. sets to zero every element of the matrix below that precission.
    btw, `ftgj` stands for fine-tuning gauss-jordan.

    Parameters
    ----------
    `coef_matrix`: array_like
        2 dimensional array with the coefficients matrix of the system
    
    `values`: array_like
        1 dimensional array with the independent values of the system

    Returns
    -------
    A tuple with the following objects:
    
        `A`: array_like
            Final reduced matrix
        
        `exp_s`, `coef_s`: int
            Both are the number of stairs that the reduced expanded and coefficients matrices have,
            respectively. At the same time, both numbers represent the ranges of their correspondig
            matrix.
            
        `n`: int
            The number of unknows (variables) of the system
    '''

    # Merge the matrix and values in one sigle matrix
    A = np.hstack((coef_matrix, values[:, np.newaxis]), dtype=float)
    m, n = coef_matrix.shape

    #We start reducing the matrix

    k, l = 0, 0     # Row and column indices

    while (k <= m-1) and (l <= n-1):
        
        print("k =", k, ", l =", l)

        # 0. Fine-Tuning

        # Set to 0 everything below our precision
        A[np.abs(A) < tol] = 0.
        print("Etapa 0:\n", A)
        
        # 1. Prepare the rows by switching accordingly

        # Select column/row 
        col = A[k:, l]
        row = A[k, l:]

        # We find the nonzero values along that column
        nonzero_idx = col.nonzero()[0]

        # Chek if the column is full of zeros
        if nonzero_idx.size == 0: 
            # Move to the next column if so and skip
            l += 1  
            continue
        
        # Find the min value along that column
        min_nonzero_idx = nonzero_idx[np.argmin(np.abs(col)[np.abs(col) != 0])] + k

        # Switch rows
        A[[k, min_nonzero_idx], l:] = A[[min_nonzero_idx, k], l:]

        # Make a 1 in the row
        A[k, l:] = row / row[0]
        print("Etapa 1:\n", A)

        # 2. Perform operations

        # Rows that do not match with the selected row
        idx = np.arange(m)
        idx = idx[idx != k]
        
        # Block build with previous rows and first coefficients 
        block = A[idx, l:]
        coef = block[:, [0]]

        # Row operations
        A[idx, l:] = block - coef * row  
        print("Etapa 2:\n", A)
        
        # 3. Move to the next iteration

        k += 1
        l += 1
    
    # Counting stairs of the expanded and coefficients matrix
    exp_s = np.unique(A.nonzero()[0]).size 
    coef_s = np.unique(A[:, :-1].nonzero()[0]).size

    # In case we have an undetermined system, we finally reduce the last column
    if exp_s != coef_s:
        idx = np.arange(m)
        A[exp_s-1, n] = 1
        A[:, n][idx != exp_s-1] = 0

    return A, n, exp_s, coef_s

Ahora, consideremos como ejemplo la siguiente matriz:

In [18]:
matrix = np.array([
    [1.234, 2.345, 3.456, 4.567],
    [5.678, 6.789, 7.890, 8.901],
    [9.012, 8.123, 0.234, 3.345],
    [4.456, 5.567, 6.678, 7.789]
], dtype=float)

Se trata de una matriz 4x4 de números con tres decimales de precisión. Si estos datos se correspondieran a medidas realizadas con esta precisión, cualquier número por debajo de 0.01 sería indistinguible de 0. Por tanto, establecemos la precisión a `tol=0.01` y estudiemos cuando se produce la normalización en cada paso:

In [19]:
ftgj_reduction(matrix[:,:-1], matrix[:,-1], tol=0.01)

k = 0 , l = 0
Etapa 0:
 [[1.234 2.345 3.456 4.567]
 [5.678 6.789 7.89  8.901]
 [9.012 8.123 0.234 3.345]
 [4.456 5.567 6.678 7.789]]
Etapa 1:
 [[1.         1.90032415 2.8006483  3.70097245]
 [5.678      6.789      7.89       8.901     ]
 [9.012      8.123      0.234      3.345     ]
 [4.456      5.567      6.678      7.789     ]]
Etapa 2:
 [[  1.           1.90032415   2.8006483    3.70097245]
 [  0.          -4.00104052  -8.01208104 -12.11312156]
 [  0.          -9.00272123 -25.00544246 -30.0081637 ]
 [  0.          -2.90084441  -5.80168882  -8.70253323]]
k = 1 , l = 1
Etapa 0:
 [[  1.           1.90032415   2.8006483    3.70097245]
 [  0.          -4.00104052  -8.01208104 -12.11312156]
 [  0.          -9.00272123 -25.00544246 -30.0081637 ]
 [  0.          -2.90084441  -5.80168882  -8.70253323]]
Etapa 1:
 [[  1.           1.90032415   2.8006483    3.70097245]
 [  0.           1.           2.           3.        ]
 [  0.          -9.00272123 -25.00544246 -30.0081637 ]
 [  0.          -

(array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 3,
 4,
 3)

La secuencia nos muestran que, cuando `k=l=2`, la _Etapa 0_ (correspondiente al paso 0 en el bucle) reduce un valor que está por debajo de la precisión a cero: esto se puede ver comparando la _Etapa 2_ del paso `k=l=1` con la _Etapa 0_ del paso `k=l=2`. Esto permite contar valores no nulos (o dentro de la precisión) correctamente cuando pasamos a la _Etapa 1_, donde hacemos uso de la función `np.nonzero()`.

**Conclusión**. Usemos `gj_reduction` si nuestros datos no tienen una precisión prefijada, y `ftgj_reduction` en cualquier otro caso. Si se usa la versión _fine-tuning_ se deberá pensar el criterio por el que se fija la precisión de los datos: muy importante, porque la solución cambiará según el valor que se decida dar.

## Estructura de las soluciones de un sistema

Las soluciones de los sistemas de ecuaciones se pueden dar en forma vectorial. Para un sistema de $n$ incógnitas $x_1, x_2, \dots, x_n$, su _vector solución_ es un vector de $n$ componentes de la forma
$$\vec{x} = (x_1, x_2, \dots, x_n)$$
tal que si nosotros sustituimos los valores $x_i$ por su solución, obtenemos una igualdad en todas sus ecuaciones. Bajo la representación vectorial de la solución de un sistema de ecuaciones, es posible encontrar una descomposición sencilla de la solución. Consideremos un sistema de $m$ ecuaciones y $n$ incógnitas inhomogéneo (caso más general):
$$\left\{\begin{aligned}
    a_{11} x_1 &+ a_{22} x_2 &+ \cdots &+ a_{1n} x_n &= b_1 \\
    a_{21} x_1 &+ a_{22} x_2 &+ \cdots &+ a_{2n} x_n &= b_2 \\
    & & \vdots \\
    a_{m1} x_1 &+ a_{m2} x_2 &+ \cdots &+ a_{mn} x_n &= b_m \\
\end{aligned}\right.$$
y llamemos su _sistema homogéneo asociado_ al sistema que se genera de anular todos sus términos independientes ($b_i = 0$). Entonces, sabemos que la solución $\vec{x} = (x_1, x_2, \dots, x_n)$ del sistema se puede escribir de la siguiente forma
$$\vec{x} = \vec{u} + \sum_{i=1}^k c_i \vec{u}_i$$
que se conoce como la _solución general_ del sistema, donde:

- $\vec{u}$ es una solución particular del sistema inhomogéneo.
- $\vec{u}_i$ son todas las soluciones independientes del sistema homogéneo asociado. En total, $k = n - r(A)$ soluciones, siendo $r(A)$ el rango de la matriz de coeficientes del sistema.
- $c_i$ son reales cualesquiera.

En general, esta descomposición de la solución se obtiene fácilmente si se tiene la matriz escalonada reducida del sistema:

- La solución particular $\vec{u}$ se consigue anulando todas las variables libres del sistema.
- Las soluciones independientes $\vec{u}_i$ se consiguen siguiendo este procedimiento sobre el sistema homogéneo:

    1. Se escoge una variable libre.
    2. Se anulan todas las variables libres menos la escogida, que toma valor 1.
    3. Se guarda la solución obtenida, que sería una de las soluciones independientes.
    4. Se repite desde el paso 1, descartando la variable seleccionada en el paso anterior.

Evidentemente, solo se tendrá solución para el sistema cuando sea compatible: es decir, cuando $n \geq r(A) = r(\bar{A})$, donde $\bar{A}$ sería la matriz ampliada.

Nuestro objetivo es dar un algoritmo que nos proporcione la solución general de un sistema (cuando sea compatible); esto es, el valor de la solución particular y sus soluciones independientes. Para conseguir esto, podemos aprovechar la función `gj_reduction` de la sección anterior, que nos daba 
- la forma de la matriz escalonada reducida
- el número de incógnitas
- el rango de la matriz de coeficientes
- y el rango de la matriz ampliada

Con la información que nos proporciona la función, podemos distinguir directamente si el sistema tiene solución o no: sencillamente comparando los rangos obtenidos. La dificultad radica entonces en conseguir las soluciones $\vec{u}$ y $\vec{u}_i$ a partir de la matriz escalonada reducida. 

### Ejemplo

Antes de dar la función que genera las soluciones directamente, consideremos primero un ejemplo:

In [2]:
# Matriz de coeficientes
coef_matrix = np.array([[1, 2, 3, 4],
                        [2, 4, 6, 8],
                        [3, 7, 9, 12],
                        [4, 8, 12, 16]])

# Términos independientes
val = np.array([5, 10, 18, 20])

# Filas e incógnitas
m, n = coef_matrix.shape

# Matriz ampliada
system = np.hstack((coef_matrix, val[:, np.newaxis]))

print("Sistema a resolver:")
print(system)


Sistema a resolver:
[[ 1  2  3  4  5]
 [ 2  4  6  8 10]
 [ 3  7  9 12 18]
 [ 4  8 12 16 20]]


Se trata de un sistema de 4 ecuaciones y 4 incógnitas. Podemos comprobar que el sistema es compatible indeterminado usando la función `gj_reduction`:

In [3]:
A, unknowns, aug_s, coef_s = lak.gj_reduction(system, give_syspar=True)

print("Sistema reducido:")
print(A)
print("Incógnitas:", n)
print("Peldaños de la matriz aumentada:", aug_s)
print("Peldaños de la matriz de coeficientes:", coef_s)

Sistema reducido:
[[ 1.  0.  3.  4. -1.]
 [ 0.  1.  0.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
Incógnitas: 4
Peldaños de la matriz aumentada: 2
Peldaños de la matriz de coeficientes: 2


Nuestro objetivo es dar la solución general de este sistema, para lo que tendremos que dar valores a las variables libres para obtener la solución particular, y todas sus soluciones independientes. 

Este problema se simplifica enormemente si encontramos antes los índices columna en las que se sitúan las variables libres y dependientes. Recordemos que las variables libres son las que no se encuentran en los peldaños de la escalera; las dependientes, en cambio, son las que se sitúan en los peldaños. Esquemáticamente, para la matriz anterior, tendríamos:

```markdown
v.l = variable libre
v.d = variable dependiente
t = términos independientes

        v.d   v.d  v.l  v.l  t
         |     |    |    |   |
      [[ 1.,  0.,  3.,  4., -1.],
       [ 0.,  1.,  0.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]]
```

Las variables dependientes se sitúan en el primer elemento no nulo de cada fila de la escalera (los peldaños). Los índices podemos encontrarlos de la siguiente forma:

In [4]:
# Índices de las variables dependientes (los peldaños)
stairs_idx = np.array([
    np.argmax(row != 0)         # Índice del primer elemento no nulo del peldaño
    for row in A[:coef_s, :-1]  # Nos movemos entre peldaños y sin la columna de términos independientes
])

#Índices de las variables libres (los que no están en los peldaños)
nostairs_idx = np.array([k for k in range(n) if k not in stairs_idx])

print("Índices de las variables dependientes:", stairs_idx)
print("Índices de las variables libres:", nostairs_idx)


Índices de las variables dependientes: [0 1]
Índices de las variables libres: [2 3]


Una vez identificados los índices, la solución particular y las soluciones independientes son directas de obtener: tan solo requiere despejar los coeficientes de las variables libres y asignar correctamente los valores de cada variable.

In [5]:

# 2. Damos la solución particular (variables libres nulas)

part_sol = np.zeros(n)
values = A[:, -1]   # Térimnos independientes

part_sol[stairs_idx] = values[:aug_s] # Asignamos los valores de cada variable

# 3. Damos las soluciones independientes (que se generan de anular todas las variables libres menos una)

indp_sol = []

for free_idx in nostairs_idx:
    sol = np.zeros(n)
    values = - A[:, free_idx]   # Despejamos la variable libre no nula

    sol[stairs_idx] = values[:aug_s]    # Asignamos los valores de cada variable
    sol[free_idx] = 1

    indp_sol.append(sol)

print("Solución particular:", part_sol)
print("Soluciones independientes:", indp_sol)


Solución particular: [-1.  3.  0.  0.]
Soluciones independientes: [array([-3., -0.,  1.,  0.]), array([-4., -0.,  0.,  1.])]


Cabe resaltar que si nuestro sistema hubiera sido compatible determinado, solo tendríamos una solución particular (sin soluciones independientes). Del mismo modo, si el sistema hubiera sido incompatible, no habría solución que dar.

### Implementación

Todos los casos de compatibilidad, y la solución general del sistema están implementado en la función `solve` del paquete `linalgkit`:

| Función | Parámetros | Retorno | Notas |
|---------|------------|---------|-------|
| `solve` | - `coef_matrix`: matriz 2D de coeficientes<br>- `values`: vector 1D de términos independientes (opcional, si no se da se asume sistema homogéneo) | - Si el sistema es incompatible: imprime advertencia y retorna None<br>- Si es compatible determinado: retorna solución particular<br>- Si es compatible indeterminado:<br>&nbsp;&nbsp;• sistema no homogéneo: retorna (solución particular, soluciones independientes)<br>&nbsp;&nbsp;• sistema homogéneo: retorna soluciones independientes | - Distingue entre sistemas incompatibles, determinados e indeterminados.<br>- Puede resolver sistemas homogéneos (sin `values`).<br>- Usa reducción de Gauss-Jordan para encontrar soluciones.<br>- Las soluciones independientes corresponden al sistema homogéneo asociado. |

Para el ejemplo que estudiamos antes tendríamos:

In [15]:
# Matriz de coeficientes
coef_matrix = np.array([[1, 2, 3, 4],
                        [2, 4, 6, 8],
                        [3, 7, 9, 12],
                        [4, 8, 12, 16]])

# Términos independientes
val = np.array([5, 10, 18, 20])

part_sol, indp_sol = lak.solve(coef_matrix, val)
print("Solución particular:", part_sol)
print("Soluciones independientes:", indp_sol)
print("Comprobación")
print(coef_matrix @ (part_sol + 2*indp_sol[0] + 3*indp_sol[1]) == val)

Compatible underdetermined system
Solución particular: [-1.  3.  0.  0.]
Soluciones independientes: (array([-3., -0.,  1.,  0.]), array([-4., -0.,  0.,  1.]))
Comprobación
[ True  True  True  True]
